In [1]:
from mlflow.tracking import MlflowClient

TRACKING_MODEL_URI = 'sqlite:///mlflow.db'
client = MlflowClient(tracking_uri=TRACKING_MODEL_URI)


In [2]:
client.get_experiment()

TypeError: MlflowClient.get_experiment() missing 1 required positional argument: 'experiment_id'

In [3]:
client.create_experiment('cool experiment')

'3'

In [71]:
from mlflow.entities import ViewType
runs = client.search_runs(experiment_ids='1',
                          filter_string="metrics.rmse < 6.8", run_view_type=ViewType.ACTIVE_ONLY, max_results=5, 
                          order_by=["metrics.rsme ASC"]
                     )

In [78]:
for run in runs:
    print(f"run id {run.info.run_id} rmse: {run.data.metrics['rmse']}")

run id 0eb7f00ef1aa4c409f70565d83cb6345 rmse: 6.742303328497426
run id f0c883ba47384456b3a51b52141ec9e7 rmse: 6.318445793399953
run id 90d27bd51dc94543ad23463c980da5f6 rmse: 6.448012843858803
run id 41a3fda045a44c1398557bffbd37f020 rmse: 6.760811090508089


In [68]:
runs

[<Run: data=<RunData: metrics={'rmse': 803.4332360022197,
  'training_mean_absolute_error': 41.22779651201669,
  'training_mean_squared_error': 3779875.228756857,
  'training_r2_score': -28269.27557363662,
  'training_root_mean_squared_error': 1944.1901215562375,
  'training_score': -28269.27557363662}, params={'C': '1.0',
  'dual': 'warn',
  'epsilon': '0.0',
  'fit_intercept': 'True',
  'intercept_scaling': '1.0',
  'loss': 'epsilon_insensitive',
  'max_iter': '1000',
  'random_state': 'None',
  'tol': '0.0001',
  'train-data-path': 'data/green_taxi_data.parquet',
  'val-data-path': 'data/green_taxi_data_02.parquet',
  'verbose': '0'}, tags={'estimator_class': 'sklearn.svm._classes.LinearSVR',
  'estimator_name': 'LinearSVR',
  'mlflow.log-model.history': '[{"run_id": "7aff4065af224c2c9eb9ef649ef09647", '
                              '"artifact_path": "model", "utc_time_created": '
                              '"2024-05-19 22:45:29.209875", "flavors": '
                            

In [73]:
import mlflow
mlflow.set_tracking_uri(TRACKING_MODEL_URI)

In [117]:
model_id = '90d27bd51dc94543ad23463c980da5f6'
model_uri= f'runs:/{model_id}/model'
model_name =  'taxi-duration'
mlflow.register_model(model_uri=model_uri, name = model_name)

Registered model 'taxi-duration' already exists. Creating a new version of this model...
Created version '4' of model 'taxi-duration'.


<ModelVersion: aliases=[], creation_timestamp=1716222489940, current_stage='None', description=None, last_updated_timestamp=1716222489940, name='taxi-duration', run_id='90d27bd51dc94543ad23463c980da5f6', run_link=None, source=('/Users/davidgillespie/projects/mlops_course/week '
 '2/course_work/mlruns/1/90d27bd51dc94543ad23463c980da5f6/artifacts/model'), status='READY', status_message=None, tags={}, user_id=None, version=4>

In [118]:
latest_versions = client.get_latest_versions(name = model_name)
for ver in latest_versions:
    print(f'{ver.version} {ver.current_stage} ')

3 Staging 
4 None 


/var/folders/s8/tqf9wz192kn1c457rshvhg440000gn/T/ipykernel_1868/3012734961.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.12.2/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name = model_name)


In [120]:
version = 4
staging = "Staging"
client.transition_model_version_stage(name = model_name, version=version, stage=staging, archive_existing_versions=False)

/var/folders/s8/tqf9wz192kn1c457rshvhg440000gn/T/ipykernel_1868/2787938495.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.12.2/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(name = model_name, version=version, stage=staging, archive_existing_versions=False)


<ModelVersion: aliases=[], creation_timestamp=1716222489940, current_stage='Staging', description=None, last_updated_timestamp=1716222496902, name='taxi-duration', run_id='90d27bd51dc94543ad23463c980da5f6', run_link=None, source=('/Users/davidgillespie/projects/mlops_course/week '
 '2/course_work/mlruns/1/90d27bd51dc94543ad23463c980da5f6/artifacts/model'), status='READY', status_message=None, tags={}, user_id=None, version=4>

In [121]:
client.update_model_version(name = model_name, version=version,description=f"This model version {version} has been promoted to {staging}" )

<ModelVersion: aliases=[], creation_timestamp=1716222489940, current_stage='Staging', description='This model version 4 has been promoted to Staging', last_updated_timestamp=1716222497481, name='taxi-duration', run_id='90d27bd51dc94543ad23463c980da5f6', run_link=None, source=('/Users/davidgillespie/projects/mlops_course/week '
 '2/course_work/mlruns/1/90d27bd51dc94543ad23463c980da5f6/artifacts/model'), status='READY', status_message=None, tags={}, user_id=None, version=4>

In [122]:
## retreive model from place

In [123]:
from sklearn.metrics import mean_squared_error
import pandas as pd

In [124]:
def read_dataframe(filepath):
    if filepath.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    else:
        df = pd.read_parquet(filepath)
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    # df['duration'] = df['duration'].dt.

    df = df[(df['duration']>=1 ) & (df['duration']<=60)]

    categorical = ['PULocationID', 'DOLocationID']
    numerical = ['trip_distance']
    df[categorical] = df[categorical].astype(str)
    return df

In [125]:
def preprocess_data(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
    numerical = ['trip_distance']
    df_dicts = df[categorical + numerical].to_dict(orient='records')
    
    return dv.transform(df_dicts)

In [134]:
def test_model(model_name:str, staging:str, X_test:pd.DataFrame, y_test:list):
    model = mlflow.pyfunc.load_model(f'models:/{model_name}/{staging}')
    y_preds = model.predict(X_test)
    return mean_squared_error(y_test, y_preds)

In [135]:
model_id = 'f0c883ba47384456b3a51b52141ec9e7'

In [136]:
preprocesser = client.download_artifacts(run_id = model_id, path = 'preprocessor', dst_path='.')

In [137]:
import pickle

In [138]:
with open('preprocessor/preprocessor.b', 'rb') as fin:
    dv = pickle.load(fin)

In [139]:
df = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-03.parquet')
X_test= preprocess_data(df, dv)
target = 'duration'
y_test = df[target].values

In [145]:
test_model(model_name, 'Production',X_test, y_test)

44.35058955740374

In [146]:
test_model(model_name, 'Staging',X_test, y_test)

/Users/davidgillespie/opt/anaconda3/envs/data/lib/python3.10/site-packages/mlflow/store/artifact/utils/models.py:32: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.12.2/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])


OSError: No such file or directory: '/Users/davidgillespie/projects/mlops_course/week 2/course_work/mlruns/1/90d27bd51dc94543ad23463c980da5f6/artifacts/model/.'

In [144]:
model_name

'taxi-duration'